<a href="https://colab.research.google.com/github/monika763/Euler_file/blob/main/Rust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
curl https://sh.rustup.rs -sSf | sh -s -- -y
source $HOME/.cargo/env
rustc --version



  stable-x86_64-unknown-linux-gnu unchanged - rustc 1.88.0 (6b00bc388 2025-06-23)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following (note the leading DOT):
. "$HOME/.cargo/env"            # For sh/bash/zsh/ash/dash/pdksh
source "$HOME/.cargo/env.fish"  # For fish
source $"($nu.home-path)/.cargo/env.nu"  # For nushell
rustc 1.88.0 (6b00bc388 2025-06-23)


info: downloading installer
warn: It looks like you have an existing rustup settings file at:
warn: /root/.rustup/settings.toml
warn: Rustup will install the default toolchain as specified in the settings file,
warn: instead of the one inferred from the default host triple.
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
warn: Updating existing toolchain, profile choice will be ignored
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'


In [ ]:
%%bash
source $HOME/.cargo/env
rm -rf euler_project
cargo new euler_project


    Creating binary (application) `euler_project` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [ ]:
euler_code = """
// Euler method to solve dy/dt = f(t, y)
pub fn euler_method(
    f: impl Fn(f64, f64) -> f64,
    y0: f64,
    t0: f64,
    t_end: f64,
    n: usize,
) -> Vec<(f64, f64)> {
    let h = (t_end - t0) / n as f64;
    let mut result = Vec::new();
    let mut t = t0;
    let mut y = y0;

    result.push((t, y));
    for _ in 0..n {
        y += h * f(t, y);
        t += h;
        result.push((t, y));
    }

    result
}
"""

with open("euler_project/src/euler.rs", "w") as f:
    f.write(euler_code)


In [ ]:
main_code = """
mod euler;
use std::fs::File;
use std::io::{BufWriter, Write};

// Define ODE: dy/dt = cos(t) - y
fn ode(t: f64, y: f64) -> f64 {
    t.cos() - y
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let t0 = 0.0;
    let t_end = 5.0;
    let y0 = 1.0;
    let steps_list = [20, 100, 1000];

    let file = File::create("euler_solutions.csv")?;
    let mut writer = BufWriter::new(file);

    writeln!(writer, "n,t,y")?;

    for &n in &steps_list {
        let result = euler::euler_method(ode, y0, t0, t_end, n);
        for (t, y) in result {
            writeln!(writer, "{},{:.4},{:.6}", n, t, y)?;
        }
    }

    println!("✅ CSV file 'euler_solutions.csv' created.");
    Ok(())
}
"""

with open("euler_project/src/main.rs", "w") as f:
    f.write(main_code)


In [ ]:
%%bash
cd euler_project
source $HOME/.cargo/env
cargo run --release


✅ CSV file 'euler_solutions.csv' created.


   Compiling euler_project v0.1.0 (/content/euler_project)
    Finished `release` profile [optimized] target(s) in 0.52s
     Running `target/release/euler_project`


In [ ]:
from google.colab import files
files.download("euler_project/euler_solutions.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
%%bash
echo 'plotters = "0.3"' >> euler_project/Cargo.toml


In [ ]:
main_code = """
mod euler;
use plotters::prelude::*;
use std::fs::File;
use std::io::{BufWriter, Write};

// ODE: dy/dt = cos(t) - y
fn ode(t: f64, y: f64) -> f64 {
    t.cos() - y
}

// Exact analytical solution
fn y_exact(t: f64) -> f64 {
    0.5 * (t.cos() + t.sin()) + 0.5 * (-t).exp()
}

fn main() -> Result<(), Box<dyn std::error::Error>> {
    let t_start = 0.0;
    let t_end = 5.0;
    let y0 = 1.0;
    let step_sizes = [20, 100, 1000];
    let colors = [&RED, &BLUE, &GREEN];

    // CSV file to store values
    let file = File::create("euler_solutions.csv")?;
    let mut writer = BufWriter::new(file);
    writeln!(writer, "n,t,y")?;

    // Plot image setup
    let root = BitMapBackend::new("euler_vs_analytical.png", (900, 600)).into_drawing_area();
    root.fill(&WHITE)?;
    let mut chart = ChartBuilder::on(&root)
        .caption("Euler vs Analytical Solution", ("sans-serif", 30))
        .margin(20)
        .x_label_area_size(40)
        .y_label_area_size(50)
        .build_cartesian_2d(t_start..t_end, -1.5..1.5)?;

    chart.configure_mesh().draw()?;

    // Plot Euler results for each n
    for (i, &n) in step_sizes.iter().enumerate() {
        let points = euler::euler_method(ode, y0, t_start, t_end, n);
        for &(t, y) in &points {
            writeln!(writer, "{},{:.4},{:.6}", n, t, y)?;
        }
        chart.draw_series(LineSeries::new(points.clone(), colors[i]))?
            .label(format!("Euler n={}", n))
            .legend(move |(x, y)| PathElement::new([(x, y), (x + 20, y)], colors[i]));
    }

    // Plot analytical/exact solution
    let exact: Vec<(f64, f64)> = (0..=1000)
        .map(|i| {
            let t = t_start + i as f64 * (t_end - t_start) / 1000.0;
            (t, y_exact(t))
        })
        .collect();
         chart.draw_series(LineSeries::new(exact, &BLACK))?
        .label("Analytical Solution")
        .legend(|(x, y)| PathElement::new([(x, y), (x + 20, y)], &BLACK));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .draw()?;

    println!("✅ Done! CSV + Plot generated.");
    Ok(())
}
"""

with open("euler_project/src/main.rs", "w") as f:
    f.write(main_code)


In [ ]:
%%bash
cd euler_project
source $HOME/.cargo/env
cargo run --release


✅ Done! CSV + Plot generated.


    Updating crates.io index
     Locking 110 packages to latest Rust 1.88.0 compatible versions
      Adding adler2 v2.0.1
      Adding android-tzdata v0.1.1
      Adding android_system_properties v0.1.5
      Adding autocfg v1.5.0
      Adding bitflags v1.3.2
      Adding bitflags v2.9.1
      Adding bumpalo v3.19.0
      Adding bytemuck v1.23.1
      Adding byteorder v1.5.0
      Adding cc v1.2.28
      Adding cfg-if v1.0.1
      Adding chrono v0.4.41
      Adding color_quant v1.1.0
      Adding core-foundation v0.9.4
      Adding core-foundation-sys v0.8.7
      Adding core-graphics v0.23.2
      Adding core-graphics-types v0.1.3
      Adding core-text v20.1.0
      Adding crc32fast v1.4.2
      Adding dirs v6.0.0
      Adding dirs-sys v0.5.0
      Adding dlib v0.5.2
      Adding dwrote v0.11.3
      Adding fdeflate v0.3.7
      Adding flate2 v1.1.2
      Adding float-ord v0.3.2
      Adding font-kit v0.14.3
      Adding foreign-types v0.5.0
      Adding foreign-types-macros v0.2.3

In [ ]:
from google.colab import files

files.download("euler_project/euler_vs_analytical.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>